In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ktrain 
from ktrain import vision as comp_vis

using Keras version: 2.2.4-tf


In [3]:
DATA_DIR = '/Users/sanchit/Documents/Projects/Datasets/fire_and_smoke_data/'

# configuration parameters 
TRAIN_DATA_DIR = '/Users/sanchit/Documents/Projects/Datasets/fire_and_smoke_data/train/'
VALIDATION_DATA_DIR = '/Users/sanchit/Documents/Projects/Datasets/fire_and_smoke_data/val/'
TEST_DATA_DIR = '/Users/sanchit/Documents/Projects/Datasets/fire_and_smoke_data/test/'
MODEL_PATH = "./models/mobilenetv2_.h5"
TRAIN_SAMPLES = 1600
VALIDATION_SAMPLES = 430
TEST_SAMPLES = 430
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64
EPOCHS = 20
LABELS = ["fire", "nofire"]
LR_LOSS_PLOT_PATH = "./models/lr_loss_plot.png"
#INIT_LR = 1e-2

### data generators
create data generators for both training and validation sets and apply data augmentation only to training generator

In [ ]:
data_aug = comp_vis.get_data_aug(horizontal_flip=True, 
                                 vertical_flip=True, 
                                 width_shift_range=0.1, 
                                 height_shift_range=0.1, 
                                 brightness_range=[0.6, 1.3], 
                                 zoom_range=0.15, fill_mode="reflect")

(train_data, val_data, preproc) = comp_vis.images_from_folder

In [ ]:
class LearningRateFinder:
    """ goal of this class is to provide a plot where effects of using a range 
    of learning rates on the loss is displayed. Some LRs are too low and some are 
    too high, therefore, with the help of this plot, we can find an optimal range of 
    LRs (minimum and maximum bounds). 

    Starting and ending LRs choosen for the plot are too low (where network is unable 
    to learn) or too large (where loss is too high). Therefore, a good range of min and 
    max LR bounds should be somewhere inside of it and that's the aim of this class. 

    A careful analysis of the plot is required to find the right bounds. At the end, 
    entire network will be then trained by using correct learning rate (max learning rate) or min and max LRs."""
    
    def __init__(self, model, stopFactor=4, beta=0.98):
        """ initializes variables for finding the learning rates 

          :param model: model for which learning rates and losses will be analyzed
          :param stopFactor: stop factor when the learning rate becomes too large to stop the model training process
          :param beta: used for averaging the loss value
          :param lrs: a list of tried LR values
          :param losses: a list of tried loss values
          :param avgLoss: average loss value over time 
          :param batchNum: current batch number 
          :param bestLoss: best low (of course lowest) found so far during training
          :param lrMult: learning rate multiplication factor 
          :weightsFile: filename to save initial (original) weights of the model """
        
        self.model = model
        self.stopFactor = stopFactor # not clear why 4? 

        self.beta = beta 
        self.lrs = [] # list of LRs which have been used already
        self.losses = [] # list of losses so far in the batch updates
        self.avgLoss = 0
        self.batchNum = 0 # current batch number/index

        self.bestLoss = 1e9 
        self.lrMult = 1
        self.weightsFile = None
        
    def reset(self):
        """ reset or re-initialize all variables from our constructor """
        self.lrs = []
        self.losses = []
        self.lrMult = 1
        self.avgLoss = 0 
        self.bestLoss = 1e9 
        self.batchNum = 0 
        self.weightsFile = None
        
    def is_data_iter(self, data):
        # define the set of class types we will check for
        iterClasses = ["NumpyArrayIterator", "DirectoryIterator", 
                     "DataFrameIterator", "Iterator", "Sequence"]
        # return whether our data is an iterator
        return data.__class__.__name__ in iterClasses
    
    def on_batch_end(self, batch, logs):
        """ responsible for setting/updating the new learning rate (LR) based on the current LR 
          and also, for recording current loss and LR. """
        # get the current LR from the model and save to a list of LRs which have been used already 
        lr = K.get_value(self.model.optimizer.lr)
        self.lrs.append(lr)
        
        # get the loss at the end of this batch, compute the average loss, smooth it and save to a list of losses 
        l = logs["loss"]
        self.avgLoss = (self.beta * self.avgLoss) + ((1 - self.beta)*l)
        smooth = self.avgLoss / (1 - (self.beta ** self.batchNum))
        self.losses.append(smooth)
        
        # compute maximum loss as a factor of best loss to stop the training 
        stopLoss = self.stopFactor * self.bestLoss 
        
        # check whether the loss has grown too large, therefore, stop the training
        if self.batchNum > 1 and smooth > stopLoss: 
            self.model.stop_training = True 
            return 
        
        # check if the best loss should be updated
        if self.batchNum == 1 or smooth < self.bestLoss: 
            self.bestLoss = smooth 

        # finally increase the LR and set it as new LR for model training 
        lr *= self.lrMult 
        K.set_value(self.model.optimizer.lr, lr)
        
    def find(self, trainData, startLR, endLR, epochs=None, 
           stepsPerEpoch=None, batchSize=32, sampleSize=2048, 
           verbose=1):
        """finds different learning rates (including optimal one) via model training 
          and at each batch update, new LR is update and saves current LR and loss. 

          :param trainData: training data either Numpy array data or data generator 
          :param startLR: starting learning rate 
          :param endLR: last learning rate 
          :param epochs: number of epochs to train for (if not value provided, it is calculate on a default sampleSize)
          :param stepsPerEpoch: total number of batch update steps per epoch 
          :sampleSize: size of training data to use when finding the optimal learning rate 
        """
        # reset class specific variables 
        self.reset()

        # determine if we are using a data generator or not 
        useGen = self.is_data_iter(trainData)

        # if we are using a generator and the steps_per_epoch is not supplied, 
        # raise an error 
        if useGen and stepsPerEpoch is None: 
            raise Exception ("Using generator without supplying steps_per_epoch")

          # if we are not using a generator then our entire dataset must already be 
          # in memory 
        elif not useGen:
            # get number of samples in the training data and then derive the number of steps_per_epoch 
            numSamples = len(trainData[0])
            stepsPerEpoch = np.ceil(numSamples / float(batchSize))

            # if number of epochs is not provided, then compute it based on a 
            # default sample size 
            if epochs is None:
                epochs = int(np.ceil(sampleSize / float(stepsPerEpoch)))

        # calculate total number of batch updates that will take place 
        numBatchUpdates = epochs * stepsPerEpoch 

        # derive the LR multiplier based on ending LR, starting LR and total 
        # number of batch updates (exponentially increase LR)
        self.lrMult = (endLR / startLR) ** (1.0 / numBatchUpdates)

        # save the model's original weights, so we can reset the weights when we are 
        # done finiding the optimal learning rates 
        self.weightsFile = tempfile.mkstemp()[1] 
        self.model.save_weights(self.weightsFile)

        # save the model's original LR and then set the new "starting" learning rate 
        origLR = K.get_value(self.model.optimizer.lr)
        K.set_value(self.model.optimizer.lr, startLR)

        # perform model training and at each batch: update LR and record current LR and loss 
        # create a callback that will be called at the end of each batch, which increases the LR and 
        # save current LR and loss
        callback = LambdaCallback(on_batch_end=lambda batch, logs: self.on_batch_end(batch, logs))

        # check if we are using a data iterator 
        if useGen:
            self.model.fit_generator(
                trainData,
                steps_per_epoch=stepsPerEpoch,
                epochs=epochs,
                verbose=verbose,
                callbacks=[callback])

        # otherwise, our entire training data is already in memory 
        else:
            self.model.fit(
                trainData[0], trainData[1],
                batch_size=batchSize,
                epochs=epochs,
                callbacks=[callback],
                verbose=verbose)

        # finally, when we are done, set back the original model's weights and LR values 
        self.model.load_weights(self.weightsFile)
        K.set_value(self.model.optimizer.lr, origLR)
    
    def plot_loss(self, skipBegin=10, skipEnd=1, title="learning rate finder"):
        """ plot learning rates versus losses diagram """
        lrs = self.lrs[skipBegin:-skipEnd]
        losses = self.losses[skipBegin:-skipEnd]

        # plot the learning rate versus loss
        plt.plot(lrs, losses)
        plt.xscale("log")
        plt.xlabel("Learning Rate")
        plt.ylabel("Loss")
        plt.title(title)

In [21]:
# create training data generator and initialize it with data augmentation methods 
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                                   horizontal_flip=True, 
                                   vertical_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   brightness_range=[0.6, 1.3], 
                                   zoom_range=0.15, fill_mode="reflect")

# and validation data generator
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=12345,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 1600 images belonging to 2 classes.
Found 430 images belonging to 2 classes.


### Create model 
we are using transfer learning for training our model, therefore, freeze the main layers of the model and attach a new custom F.C. classifier on the top of it.

In [22]:
# define a model 
def create_model():
    base_model = MobileNetV2(include_top=False, 
                              input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    
    for layer in base_model.layers[:]:
        layer.trainable = False
    
    input_layer = base_model.output 
    x = GlobalAveragePooling2D()(input_layer)
    x = Dense(128, activation='selu')(x)
    x = Dropout(0.5)(x)
    x = Dense(NUM_CLASSES, name="last_dense")(x)
    output = Activation("softmax", name="last_softmax")(x)
    #output = Dense(NUM_CLASSES, activation='softmax')(x)
    
    # create the final model 
    model = Model(inputs = base_model.input, outputs = output)
    model.summary()
    return model 

### Build and compile model

In [23]:
model = create_model()

#model.compile(loss='categorical_crossentropy',
#              optimizer=tf.keras.optimizers.Adam(0.001),
#              metrics=['acc'])

#opt = tf.keras.optimizers.SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / EPOCHS)
opt = tf.keras.optimizers.SGD(momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc'])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

### Train model

In [24]:
class LearningRateFinder:
    """ goal of this class is to provide a plot where effects of using a range 
    of learning rates on the loss is displayed. Some LRs are too low and some are 
    too high, therefore, with the help of this plot, we can find an optimal range of 
    LRs (minimum and maximum bounds). 

    Starting and ending LRs choosen for the plot are too low (where network is unable 
    to learn) or too large (where loss is too high). Therefore, a good range of min and 
    max LR bounds should be somewhere inside of it and that's the aim of this class. 

    A careful analysis of the plot is required to find the right bounds. At the end, 
    entire network will be then trained by using correct learning rate (max learning rate) or min and max LRs."""
    
    def __init__(self, model, stopFactor=4, beta=0.98):
        """ initializes variables for finding the learning rates 

          :param model: model for which learning rates and losses will be analyzed
          :param stopFactor: stop factor when the learning rate becomes too large to stop the model training process
          :param beta: used for averaging the loss value
          :param lrs: a list of tried LR values
          :param losses: a list of tried loss values
          :param avgLoss: average loss value over time 
          :param batchNum: current batch number 
          :param bestLoss: best low (of course lowest) found so far during training
          :param lrMult: learning rate multiplication factor 
          :weightsFile: filename to save initial (original) weights of the model """
        
        self.model = model
        self.stopFactor = stopFactor # not clear why 4? 

        self.beta = beta 
        self.lrs = [] # list of LRs which have been used already
        self.losses = [] # list of losses so far in the batch updates
        self.avgLoss = 0
        self.batchNum = 0 # current batch number/index

        self.bestLoss = 1e9 
        self.lrMult = 1
        self.weightsFile = None
        
    def reset(self):
        """ reset or re-initialize all variables from our constructor """
        self.lrs = []
        self.losses = []
        self.lrMult = 1
        self.avgLoss = 0 
        self.bestLoss = 1e9 
        self.batchNum = 0 
        self.weightsFile = None
        
    def is_data_iter(self, data):
        # define the set of class types we will check for
        iterClasses = ["NumpyArrayIterator", "DirectoryIterator", 
                     "DataFrameIterator", "Iterator", "Sequence"]
        # return whether our data is an iterator
        return data.__class__.__name__ in iterClasses
    
    def on_batch_end(self, batch, logs):
        """ responsible for setting/updating the new learning rate (LR) based on the current LR 
          and also, for recording current loss and LR. """
        # get the current LR from the model and save to a list of LRs which have been used already 
        lr = K.get_value(self.model.optimizer.lr)
        self.lrs.append(lr)
        
        # get the loss at the end of this batch, compute the average loss, smooth it and save to a list of losses 
        l = logs["loss"]
        self.avgLoss = (self.beta * self.avgLoss) + ((1 - self.beta)*l)
        smooth = self.avgLoss / (1 - (self.beta ** self.batchNum))
        self.losses.append(smooth)
        
        # compute maximum loss as a factor of best loss to stop the training 
        stopLoss = self.stopFactor * self.bestLoss 
        
        # check whether the loss has grown too large, therefore, stop the training
        if self.batchNum > 1 and smooth > stopLoss: 
            self.model.stop_training = True 
            return 
        
        # check if the best loss should be updated
        if self.batchNum == 1 or smooth < self.bestLoss: 
            self.bestLoss = smooth 

        # finally increase the LR and set it as new LR for model training 
        lr *= self.lrMult 
        K.set_value(self.model.optimizer.lr, lr)
        
    def find(self, trainData, startLR, endLR, epochs=None, 
           stepsPerEpoch=None, batchSize=32, sampleSize=2048, 
           verbose=1):
        """finds different learning rates (including optimal one) via model training 
          and at each batch update, new LR is update and saves current LR and loss. 

          :param trainData: training data either Numpy array data or data generator 
          :param startLR: starting learning rate 
          :param endLR: last learning rate 
          :param epochs: number of epochs to train for (if not value provided, it is calculate on a default sampleSize)
          :param stepsPerEpoch: total number of batch update steps per epoch 
          :sampleSize: size of training data to use when finding the optimal learning rate 
        """
        # reset class specific variables 
        self.reset()

        # determine if we are using a data generator or not 
        useGen = self.is_data_iter(trainData)

        # if we are using a generator and the steps_per_epoch is not supplied, 
        # raise an error 
        if useGen and stepsPerEpoch is None: 
            raise Exception ("Using generator without supplying steps_per_epoch")

          # if we are not using a generator then our entire dataset must already be 
          # in memory 
        elif not useGen:
            # get number of samples in the training data and then derive the number of steps_per_epoch 
            numSamples = len(trainData[0])
            stepsPerEpoch = np.ceil(numSamples / float(batchSize))

            # if number of epochs is not provided, then compute it based on a 
            # default sample size 
            if epochs is None:
                epochs = int(np.ceil(sampleSize / float(stepsPerEpoch)))

        # calculate total number of batch updates that will take place 
        numBatchUpdates = epochs * stepsPerEpoch 

        # derive the LR multiplier based on ending LR, starting LR and total 
        # number of batch updates (exponentially increase LR)
        self.lrMult = (endLR / startLR) ** (1.0 / numBatchUpdates)

        # save the model's original weights, so we can reset the weights when we are 
        # done finiding the optimal learning rates 
        self.weightsFile = tempfile.mkstemp()[1] 
        self.model.save_weights(self.weightsFile)

        # save the model's original LR and then set the new "starting" learning rate 
        origLR = K.get_value(self.model.optimizer.lr)
        K.set_value(self.model.optimizer.lr, startLR)

        # perform model training and at each batch: update LR and record current LR and loss 
        # create a callback that will be called at the end of each batch, which increases the LR and 
        # save current LR and loss
        callback = LambdaCallback(on_batch_end=lambda batch, logs: self.on_batch_end(batch, logs))

        # check if we are using a data iterator 
        if useGen:
            self.model.fit_generator(
                trainData,
                steps_per_epoch=stepsPerEpoch,
                epochs=epochs,
                verbose=verbose,
                callbacks=[callback])

        # otherwise, our entire training data is already in memory 
        else:
            self.model.fit(
                trainData[0], trainData[1],
                batch_size=batchSize,
                epochs=epochs,
                callbacks=[callback],
                verbose=verbose)

        # finally, when we are done, set back the original model's weights and LR values 
        self.model.load_weights(self.weightsFile)
        K.set_value(self.model.optimizer.lr, origLR)
    
    def plot_loss(self, skipBegin=10, skipEnd=1, title="learning rate finder"):
        """ plot learning rates versus losses diagram """
        lrs = self.lrs[skipBegin:-skipEnd]
        losses = self.losses[skipBegin:-skipEnd]

        # plot the learning rate versus loss
        plt.plot(lrs, losses)
        plt.xscale("log")
        plt.xlabel("Learning Rate")
        plt.ylabel("Loss")
        plt.title(title)

In [ ]:
lrf = LearningRateFinder(model)
lrf.find(
    train_generator, 
    startLR=1e-10, 
    endLR=1e+1, 
    epochs=15,
    stepsPerEpoch=TRAIN_SAMPLES // BATCH_SIZE, 
    batchSize=BATCH_SIZE)

# plot the loss for the various learning rates and save the
# resulting plot to disk
lrf.plot_loss()
plt.savefig(LR_LOSS_PLOT_PATH)

# gracefully exit the script so we can adjust our learning rates
# in the config and then train the network for our full set of
# epochs
print("[INFO] learning rate finder complete")
print("[INFO] examine plot and adjust learning rates before training")
sys.exit(0)

  ...
    to  
  ['...']
Train for 25 steps
Epoch 1/15
 1/25 [>.............................] - ETA: 1:43 - loss: 1.0290 - acc: 0.5312

/anaconda3/envs/tf_venv_2.1/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in double_scalars


23/25 [==========================>...] - ETA: 4s - loss: 1.1056 - acc: 0.4980

/anaconda3/envs/tf_venv_2.1/lib/python3.7/site-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


25/25 [==============================] - 60s 2s/step - loss: 1.0969 - acc: 0.4988
Epoch 2/15
23/25 [==========================>...] - ETA: 4s - loss: 1.0370 - acc: 0.5102

SystemExit: 0

In [ ]:
# define callbacks before starting the training
early_stop = EarlyStopping(monitor="val_loss", patience=10, mode="min", verbose=1)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=0.00001, verbose=1)
model_checkpoint = ModelCheckpoint(MODEL_PATH, monitor="val_acc", save_best_only=True, mode='max', verbose=1)
callbacks = [reduce_lr, early_stop, model_checkpoint]

# perform model training 
H = model.fit_generator(generator=train_generator, 
                    steps_per_epoch=TRAIN_SAMPLES // BATCH_SIZE, 
                    epochs=EPOCHS,
                    validation_data=validation_generator,
                    validation_steps=VALIDATION_SAMPLES // BATCH_SIZE, 
                    callbacks=callbacks)

### Evaluate results 
Plot Confusion matrix and classification report on both validation and test samples

In [ ]:
def get_mismatches(y_true: list, y_pred: list, filenames: list):
    """ return a list of wrong predictions (or, mis-matches) done by the trained model 
        :param y_true: a list of true (or ground-truth) labels 
        :param y_pred: a list of predicted labels 
        :param filenames: a list of filenames corresponding to the true labels
    """
    mismatches_imgs = list(dict())
    for i in range(0, len(y_true)):
        if y_true[i] != y_pred[i]:
            print(f"True class: {LABELS[y_true[i]]}({y_true[i]}) and predicted as: {LABELS[y_pred[i]]}({y_pred[i]})")
            temp = dict()
            temp["filename"] = filenames[i]
            temp["true label"] = y_true[i]
            temp["pred label"] = y_pred[i]
            mismatches_imgs.append(temp)
        continue
    return mismatches_imgs

In [ ]:
# Plot confusion matrix, classification report and mis-matches information for the validation data
y_pred = model.predict_generator(validation_generator, steps=VALIDATION_SAMPLES // BATCH_SIZE + 1)
y_pred = np.argmax(y_pred, axis=1)
print("Confusion matrix:")
print(confusion_matrix(y_true=validation_generator.classes, y_pred=y_pred))
print("-"*50)
print("Classification report:")
print(classification_report(y_true=validation_generator.classes, y_pred=y_pred, target_names=LABELS))
print("Mismatches information:")
print(get_mismatches(validation_generator.classes, y_pred, validation_generator.filenames))

In [ ]:
# Similarly, plot confusion for the test dataset
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    TEST_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

In [ ]:
# Plot confusion matrix, classification report and mis-classification (or, mis-matches) information for the test data
y_pred = model.predict_generator(test_generator, steps=TEST_SAMPLES // BATCH_SIZE + 1)
y_pred = np.argmax(y_pred, axis=1)
print("Confusion matrix:")
print(confusion_matrix(y_true=test_generator.classes, y_pred=y_pred))
print("-"*50)
print("Classification report:")
print(classification_report(y_true=test_generator.classes, y_pred=y_pred, target_names=LABELS))
print("Mismatches information:")
print(get_mismatches(test_generator.classes, y_pred, validation_generator.filenames))